# How do organisms respond to oxidative stress?

### Analysis 1
* PQ is applied to Wild-type worms
* Fastq: Con_N2 vs PQ_N2

### Analysis 2
* PQ is applied to e113 Worms
* Fastq: Con_e113 vs PQ_e113

### Analysis 4
* Comparing Wild Type and e113 untreated
* Fastq: Con_N2 vs Con_e113

### Analysis 5
* Comparing PQ treated to Wild Type and e113
* Fastq: PQ_N2 vs PQ_e113




In [13]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter


from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3

import seaborn as sns
import umap
import matplotlib.pyplot as plt

In [14]:
input_dir = "./input_data"
output_dir = "./output_data"

wormcat_df = pd.read_csv(f"{input_dir}/whole_genome_v2_nov-11-2021.csv") 


analysis_df  = {}
for direction in ['up', 'down']:
    for analysis in [1, 2, 4, 5]:
        df = pd.read_csv(f"{input_dir}/analysis{analysis}_{direction}-with-correction.csv") 
        analysis_df[f"{direction}_A{analysis}"] = df
        
#analysis_df

In [15]:
def highlight_rows_by_column(worksheet, column_number, color):
    color_fill = PatternFill(start_color=color, end_color=color, fill_type='solid')
    df = pd.DataFrame(worksheet.values)
    for row_idx, row in enumerate(df.itertuples(), start=1):
        if isinstance(row[column_number], bool) and row[column_number] is True:  # Check if the column value is True
            for col_idx in range(1, len(row)):
                worksheet.cell(row=row_idx, column=col_idx).fill = color_fill

def autofit_columns(worksheet):
    for column in worksheet.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2) * 1.2  # Adding some extra padding
        worksheet.column_dimensions[get_column_letter(column[0].column)].width = adjusted_width

def venn_plot(left, right, overlap_set1_set2, label, filename):
    label_parts = label.split('_')
    analysis_name_map={'A1':'Con_N2 - PQ_N2','A2':'Con_e113 - PQ_e113','A4':'Con_N2 - Con_e113','A5':'PQ_N2 - PQ_e113'}
    labels = [analysis_name_map[label_parts[1]],analysis_name_map[label_parts[2]]]
    plt.figure(figsize=(7,7))
    ax = plt.gca()
    v= venn2(subsets=[len(left),len(right),len(overlap_set1_set2)], set_labels=labels, ax=ax,set_colors=('darkviolet','deepskyblue'),alpha=0.5)

    # Save the plot
    plt.title(f"{label_parts[0].upper()} Regulated Genes across\n    {analysis_name_map[label_parts[1]]} and {analysis_name_map[label_parts[2]]}", fontdict={'fontsize': 18,'fontweight': 'bold'})
    plt.savefig(filename)
    plt.close()
    return

def get_value(wormcat_df, key, column_name):
    value_at  = wormcat_df.loc[wormcat_df['Wormbase ID'] == key, column_name]
    value = ""
    if not value_at.empty:
        value = value_at.iloc[0]
    return value

In [16]:
match_ups = [('A1','A2'),('A1','A4'),('A1','A5'),('A2','A4'),('A2','A5'),('A4','A5')]
match_up_dfs = {}
for direction in ['up', 'down']:  
    for match_up in match_ups:
        analysis1_df =analysis_df[f"{direction}_{match_up[0]}"]
        analysis2_df =analysis_df[f"{direction}_{match_up[1]}"]
        set1 = set(analysis1_df['ID'])
        set2 = set(analysis2_df['ID'])
        union_set1_set2 = set1.union(set2)
        intersect = set1.intersection(set2)
        left = set1 - intersect
        right = set2 - intersect
        data_dict = {}
        for key in union_set1_set2:
            value = {'left_only'  : True if key in left else False,
                     'right_only' : True if key in right else False,
                     'intersect'  : True if key in intersect else False,
                     'sequence_id': get_value(wormcat_df, key, 'Sequence ID'),
                     'category'   : get_value(wormcat_df, key, 'Category 1')
                    }
            data_dict[key]=value

        df = pd.DataFrame.from_dict(data_dict, orient='index')
        
        label = f"{direction}_{match_up[0]}_{match_up[1]}"
        match_up_dfs[label]=df
        filename = f"{output_dir}/{label}.png"
        venn_plot(left, right, intersect, label, filename)
        
#match_up_dfs        

In [17]:
match_up_dfs['up_A1_A2']

,left_only,right_only,intersect,sequence_id,category
WBGene00008850,False,False,True,F15B9.6,Transmembrane protein
WBGene00016119,False,True,False,C25H3.10,Unassigned
WBGene00007531,True,False,False,C11H1.7,Extracellular material
WBGene00004926,True,False,False,C08G5.4,Neuronal function
WBGene00001462,True,False,False,M79.4,Neuronal function
...,...,...,...,...,...
WBGene00007345,True,False,False,C05E7.3,Unassigned
WBGene00022416,True,False,False,Y102A11A.6,Trafficking
WBGene00044457,False,True,False,C18H7.11,Transmembrane protein
WBGene00008577,False,False,True,F08G2.5,Stress response


In [18]:
# Put it all together and generate the output
output_dir="./output_data"
excel_file_path = f"{output_dir}/compare_across_experiments.xlsx"
if os.path.exists(excel_file_path):
    os.remove(excel_file_path)

# Create a new workbook
workbook = Workbook()

# Remove the default "Sheet" created by openpyxl
default_sheet = workbook['Sheet']
workbook.remove(default_sheet)

with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    writer.book = workbook
    for sheet_name in match_up_dfs.keys():
        match_up_dfs[sheet_name] = match_up_dfs[sheet_name].sort_values(by=['left_only', 'right_only', 'intersect', 'category'], ascending=[False, False, False, True])
        match_up_dfs[sheet_name].to_excel(writer, sheet_name=sheet_name, index=True, index_label='ID')
        
        
    # Autofit and highlight columns for each sheet
    for sheet in writer.sheets.values():
        autofit_columns(sheet)
        for index, color in  enumerate(['ca7fe8','7fdeff','b3c2ff']):
            highlight_rows_by_column(sheet,index+2, color)
            
         
        